# imports

In [ ]:
import os
import json
import dateutil
import datetime
import SensorThings as st
import numpy as np
import geopandas as gpd
import pandas as pd
from shapely.geometry import LineString
from shapely.geometry import Point
import folium
import branca.colormap as cm

# settings

In [ ]:
reloaddata = True

st_url = "http://smartaqnet.teco.edu:8080/FROST-Server"
datastream_id = "testdatastream:80:7D:3A:DA:CF:3E"

starttime = "2018-11-13T00:00:00.000Z"
endtime = "2018-11-13T23:59:59.999Z"

# load data

In [ ]:
qdata = [
    "$expand=" +
        "Datastream($select=id)," +
        "FeatureOfInterest($select=id,feature)",

    "$filter=" +
        "phenomenonTime ge " + starttime + " and "
        "phenomenonTime le " + endtime,

    "$select=id,result,phenomenonTime"
]

def load_data(filename, where):
    if (not os.path.isfile(filename)) or reloaddata:
        entities = st.GetEntities(st_url, where, qdata)
        with open(filename, 'w') as outfile:
            json.dump(entities, outfile)
    else:
        with open(filename, 'r') as infile:
            entities = json.load(infile)
            
    data = [
        [
            dateutil.parser.parse(elem['phenomenonTime']).timestamp(),
            *(elem['FeatureOfInterest']['feature']['coordinates']),
            elem["result"],
            elem["@iot.id"],
            elem["FeatureOfInterest"]["@iot.id"],
            elem["Datastream"]["@iot.id"],
        ]
        for elem in entities
    ]
    data.sort(key=lambda x: x[0])
    data_cols = list(zip(*data))
    gdf = gpd.GeoDataFrame(
        {
            "timestamp": data_cols[0],
            "longitude": data_cols[1],
            "latitude": data_cols[2],
            "elevation": data_cols[3],
            "value": data_cols[4],
            "@iot.id": data_cols[5],
            "FeatureOfInterest.@iot.id": data_cols[6],
            "Datastream.@iot.id": data_cols[7],
        },
        geometry = [Point(xy) for xy in zip(*data_cols[1:3])],
        crs = {'init': 'epsg:4326'}
    )
    
    
    return gdf


# PM10 data
filename = "entities_pm10.json"
where = "/v1.0/Datastreams('" + datastream_id + ":PM10')/Observations"
PM10 = load_data(filename, where)

# PM2.5 data
filename = "entities_pm25.json"
where = "/v1.0/Datastreams('" + datastream_id + ":PM25')/Observations"
PM25 = load_data(filename, where)

# calculate differences

In [ ]:
def calc_LineStrings(gdf):
    for i in range(len(gdf) - 1):
        geom0 = gdf.loc[i]["geometry"]
        geom1 = gdf.loc[i + 1]["geometry"]
    
        start, end = [(geom0.x, geom0.y), (geom1.x, geom1.y)]
        line = LineString([start, end])
        yield line
        
        
def calc_timedelta(gdf):
    temp = gdf[1:].copy()
    temp.index = range(len(temp))
    timedelta = temp["timestamp"] - gdf["timestamp"]
    
    return timedelta
    
    
def calc_distance(gdf):
    temp = gpd.GeoDataFrame(
        gdf.copy()[:-1],
        geometry = list(calc_LineStrings(gdf)),
        crs = {'init': 'epsg:4326'}
    )
    temp.index = range(len(temp))
    distance = temp.length
    
    return distance
    
        
def calc_differences(gdf):
    gdf_out = gdf.copy()
    
    # calculate initial stuff
    gdf_out.loc[:, "timedelta"] = calc_timedelta(gdf)
    gdf_out.loc[:, "distance"] = calc_distance(gdf)
    
    # drop some stuff
    gdf_out = gdf_out[gdf_out["distance"] != 0.0]
    gdf_out = gdf_out[gdf_out["distance"] < 0.5*10e-4]
    gdf_out = gdf_out[gdf_out["timedelta"] > 0.1]
    gdf_out.index = range(len(gdf_out))
    
    # calculate final stuff
    gdf_out.loc[:, "timedelta"] = calc_timedelta(gdf_out)
    gdf_out = gpd.GeoDataFrame(
        gdf_out[:-1],
        geometry = list(calc_LineStrings(gdf_out)),
        crs = {'init': 'epsg:4326'}
    )
    gdf_out.loc[:, "distance"] = gdf_out.length
    gdf_out.loc[:, "velocity"] = gdf_out["distance"].divide(gdf_out["timedelta"])
    
    return gdf_out


PM10_diff = calc_differences(PM10)
PM25_diff = calc_differences(PM25)

# map

In [ ]:
fig = folium.Figure(height = 400)

map_center = (PM10["latitude"].mean(), PM10["longitude"].mean())
m = folium.Map(location = map_center, zoom_start = 14, tiles = "Cartodb dark_matter")

plotgdf = PM10_diff
plotcol = "value"

linear = cm.linear.YlOrRd_03.scale(plotgdf[plotcol].min(), plotgdf[plotcol].max()/10.0)
layer = folium.GeoJson(plotgdf,
                       style_function = lambda feature: {
                           'color': linear(feature["properties"][plotcol]),
                           'weight': 3
                       })

layer.add_child
    
m.add_child(linear)
m.add_child(layer)
m.add_to(fig)